In [1]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai
import re

# load environment variables
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/prompts/")

from vqa_question.v1 import prompt_template as vqa_prompt_template
from drone_explore_prompt import prompt_template as explore_template
from drone_next_action.v1 import prompt_template as drone_prompt_template

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Scripts/")

from data_collection_utils import take_individual_picture, stream_video, stream_frames, record_streamed_frames

print("Imported all modules")


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imported all modules


In [21]:
from djitellopy import Tello

In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
drone = Tello()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.


In [36]:
drone.TELLO_IP

'192.168.10.1'

In [33]:
drone.get_distance_tof()

KeyError: '192.168.10.1'

In [16]:
for e in list(dir(drone))[45:]:
    print(e)

__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
address
background_frame_read
cap
connect
connect_to_wifi
curve_xyz_speed
curve_xyz_speed_mid
disable_mission_pads
emergency
enable_mission_pads
end
flip
flip_back
flip_forward
flip_left
flip_right
get_acceleration_x
get_acceleration_y
get_acceleration_z
get_barometer
get_battery
get_current_state
get_distance_tof
get_flight_time
get_frame_read
get_height
get_highest_temperature
get_lowest_temperature
get_mission_pad_distance_x
get_mission_pad_distance_y
get_mission_pad_distance_z
get_mission_pad_id
get_own_udp_object
get_pitch
get_roll
get_speed_x
get_speed_y
get_speed_z
get_state_field
get_temperature
get_udp_video_address
get_video_capture
get_yaw
go_xyz_speed
go_xyz_speed_mid
go_xyz_speed_yaw_mid
initiate_throw_takeoff
is_flying
land
last_rc_control_timestamp
last_received_command_timestamp
move
move_back
move_down
move_forward
move_left
move_right
move_up


In [3]:
drone.get_battery()

Exception: Could not get state property: bat

In [4]:
objective = "Find the person who is wearing a blue cap"

In [6]:
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'


OSError: [Errno 51] Network is unreachable

In [5]:
drone.streamon()
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds


Exception: Command 'streamon' was unsuccessful for 4 tries. Latest response:	'Aborting command 'streamon'. Did not receive a response after 7 seconds'

In [6]:
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'takeoff'


KeyboardInterrupt: 

In [32]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [33]:
# Generate next action prompt
def next_action_prompt(prompt_template, objective, environment_context, previous_commands, previous_context, vqa_questions):
    prompt = prompt_template.replace("$objective", objective)
    prompt = prompt.replace("$context", environment_context)
    prompt = prompt.replace("$previous_commands", previous_commands)
    prompt = prompt.replace("$previous_context", previous_context)
    prompt = prompt.replace("$vqa_questions", vqa_questions)
    return prompt, ["\n"] # prompt and stop token

In [34]:
def prompt(prompt, stop_tokens):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.5,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=10,
        stop=stop_tokens,
    )
    return response.choices[0].text

In [35]:
def get_vqa_questions(vqa_prompt_template, objective):
    prompt_text = vqa_prompt_template.replace("$objective", objective)
    result = prompt(prompt_text, ["&&&&&&"])
    matches = re.findall(r'@(.+?)@', result)
    return matches


In [36]:
get_vqa_questions(vqa_prompt_template, objective)

['Is there a person in the image?', 'Is there a blue cap in the image?']

In [ ]:
def explore(drone, use_gpt=False, prompt_template=explore_template):
    if use_gpt:
       prompt(prompt_template)
    else:
        import random as r
        # randomly generate a number between 45 and 180
        random_angle = r.randint(45, 180)
        # randomly generate a number between 5 and 20
        random_distance = r.randint(5, 20)
        actions = [(drone.move_left, random_distance), (drone.move_right, random_distance), (drone.rotate_clockwise, random_angle), (drone.rotate_counter_clockwise, random_angle), (drone.move_forward, random_distance), (drone.move_back, random_distance)]

        # sample 3 actions
        sampled_actions = r.sample(actions, 3)
        for action in sampled_actions:
            action[0](action[1])
            time.sleep(0.5)
